<a href="https://colab.research.google.com/github/RiddhimanRaut/My_Drake_Adventure/blob/main/My_pick_place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# TODO(russt): install only drake (not manipulation) after colab gets to python3.8
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='c1bdae733682f8a390f848bc6cb0dbbf9ea98602', drake_version='0.27.0', drake_build='release')

from manipulation import running_as_notebook

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


# Let's do all of our imports here, too.
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG
from ipywidgets import Text, Textarea, Layout
import plotly.express as px

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

# TODO(russt): Move this to drake (adding the element name support to the base class).
import pandas as pd

def dataframe(trajectory, times, names):
  assert trajectory.rows() == len(names)
  values = trajectory.vector_values(times)
  data = {'t': times }
  for i in range(len(names)):
    data[names[i]] = values[i,:]
  return pd.DataFrame(data)

Cloning into '/opt/manipulation'...

HEAD is now at c1bdae7 make running_as_notebook a util, and add colab testing logic

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'notebook.nbextensions' found in sys.modules after import of package 'notebook', but prior to execution of 'notebook.nbextensions'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

Reason for being yanked: Name generation process created inappropriate id values
ERROR: pip's dependency resolver does not currently take into account all the packages tha

Just a brick in the whole wide world.

In [8]:
def my_brick():
  plant = MultibodyPlant(time_step = 0.0)
  parser = Parser(plant)
  parser.AddModelFromFile(FindResourceOrThrow(
      "drake/examples/manipulation_station/models/061_foam_brick.sdf"
  ))
  plant.Finalize()
  #context
  context = plant.CreateDefaultContext()
  print(context)

  print(f"Brick Positions (m): {plant.num_positions()}")
  print(f"Brick Velocities (m/s and rad/s): {plant.num_velocities()}")

my_brick()

::_ Context
------------
Time: 0
States:
  13 continuous states
    1 0 0 0 0 0 0 0 0 0 0 0 0

Parameters:
  3 numeric parameter groups with
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
             0.028           0           0       0.025  0.00104286  0.00130357 0.000678571           0           0           0
     12 parameters
        1  0  0  0  1  0  0  0  1 -0 -0 -0

Brick Positions (m): 7
Brick Velocities (m/s and rad/s): 6


Forward Kinematics:

In [21]:
def forward_kinematics_example():
  builder = DiagramBuilder()
  #creating station
  station = builder.AddSystem(ManipulationStation())
  station.SetupClutterClearingStation()
  station.Finalize()

  frames_to_draw = {"iiwa":{"iiwa_link_1","iiwa_link_2","iiwa_link_3","iiwa_link_4","iiwa_link_5","iiwa_link_6","iiwa_link_7"},
                    "gripper":{"body"}}
  meshcat = ConnectMeshcatVisualizer(builder, 
                                     station.get_scene_graph(),
                                     output_port = station.GetOutputPort("query_object"),
                                     zmq_url = zmq_url,
                                     frames_to_draw = frames_to_draw,
                                     axis_length = 0.3,
                                     axis_radius = 0.01)
  meshcat.load()
  diagram = builder.Build()
  context = diagram.CreateDefaultContext()
  xyz = Text(value = "", description = "Gripper positions (m): ", layout = Layout(width = '500px', style = 'description_width:initial'))
  rpy = Text(value = "", description = "Gripper orientation (rad): ", layout = Layout(width = '500px', style = 'description_width:initial'))

  plant = station.get_multibody_plant()
  gripper = plant.GetBodyByName("body")

  def pose_callback(context):
    pose = plant.EvalBodyPoseInWorld(context, gripper)
    xyz.value = np.array2string(pose.translation())
    rpy.value = np.array2string(RollPitchYaw(pose.rotation()).vector())
  
  MakeJointSlidersThatPublishOnCallback(plant, meshcat, context, pose_callback)
  display(xyz)
  display(rpy)
forward_kinematics_example()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://9530bf3f282f.ngrok.io/static/
Connected to meshcat-server.


FloatSlider(value=-1.57, description='iiwa_joint_1', layout=Layout(width="'200'"), max=2.96706, min=-2.96706, …

FloatSlider(value=0.1, description='iiwa_joint_2', layout=Layout(width="'200'"), max=2.0944, min=-2.0944, step…

FloatSlider(value=0.0, description='iiwa_joint_3', layout=Layout(width="'200'"), max=2.96706, min=-2.96706, st…

FloatSlider(value=-1.2, description='iiwa_joint_4', layout=Layout(width="'200'"), max=2.0944, min=-2.0944, ste…

...

FloatSlider(value=0.0, description='iiwa_joint_7', layout=Layout(width="'200'"), max=3.05433, min=-3.05433, st…

FloatSlider(value=-0.05, description='left_finger_sliding_joint', layout=Layout(width="'200'"), max=0.0, min=-…

FloatSlider(value=0.05, description='right_finger_sliding_joint', layout=Layout(width="'200'"), max=0.055, ste…

Text(value='[ 3.75355714e-04 -4.71358790e-01  6.56018583e-01]', description='Gripper positions (m): ', layout=…

Text(value='[-1.81238898e+00 -1.86850448e-16  7.96326795e-04]', description='Gripper orientation (rad): ', lay…

Computing the Jacobian.